In [1]:
import os
# path = os.getcwd()+'/graphs/sccs'
# variable =''
# for file in os.listdir(path):
#     f = open(path +'/'+file, 'r')
#     variable += file +' '+f.readline()
# variable = variable.replace(' ',',')
#
# with open("meta.csv", "w") as text_file:
#     text_file.write(variable)

In [2]:
import pandas as pd

df_m =pd.read_csv(r'meta.csv')
df_a = pd.read_csv(r'all.csv')
df_a['graph_name'] = df_a['graph_name'].astype(str)
df_m['graph_name'] = df_m['graph_name'].astype(str)

In [3]:
x=pd.merge(df_a,df_m, left_on='graph_name', right_on='graph_name')
x = x.groupby(['graph_name', 'mcts_time'], as_index=False).agg('mean')

In [4]:
x['diff'] = x['v']-x['mdfvs_size']
x.sort_values(by='diff', ascending=True)

,graph_name,mcts_time,time,mdfvs_size,v,e,diff
103,3272_e_077,5,15.266695,31.0,43.0,268.0,12.0
160,3758_e_081,0,20.146496,28.0,40.0,290.0,12.0
161,3758_e_081,1,20.088540,28.0,40.0,290.0,12.0
162,3758_e_081,2,22.808627,28.0,40.0,290.0,12.0
163,3758_e_081,5,28.216885,28.0,40.0,290.0,12.0
...,...,...,...,...,...,...,...
35,1979_e_057,0,32.138528,69.0,110.0,426.0,41.0
39,1979_e_057,10,41.842294,69.0,110.0,426.0,41.0
38,1979_e_057,5,34.537217,69.0,110.0,426.0,41.0
37,1979_e_057,2,29.051349,69.0,110.0,426.0,41.0


In [5]:
import numpy as np
y = pd.DataFrame()
y['graph_name'] = x['graph_name'].unique()
y['time_0'] = np.nan
y['time_1'] = np.nan
y['time_2'] = np.nan
y['time_5'] = np.nan
y['time_10'] = np.nan

for i, row in y.iterrows():
    y.loc[y['graph_name'] == row['graph_name'],['time_0']] = x.loc[(x['graph_name']==row['graph_name']) & (x['mcts_time'] == 0), ['time']].values[0]
    y.loc[y['graph_name'] == row['graph_name'],['time_1']] = x.loc[(x['graph_name']==row['graph_name']) & (x['mcts_time'] == 1), ['time']].values[0]
    y.loc[y['graph_name'] == row['graph_name'],['time_2']] = x.loc[(x['graph_name']==row['graph_name']) & (x['mcts_time'] == 2), ['time']].values[0]
    y.loc[y['graph_name'] == row['graph_name'],['time_5']] = x.loc[(x['graph_name']==row['graph_name']) & (x['mcts_time'] == 5), ['time']].values[0]
    y.loc[y['graph_name'] == row['graph_name'],['time_10']] = x.loc[(x['graph_name']==row['graph_name']) & (x['mcts_time'] == 10), ['time']].values[0]

In [6]:
x = x.groupby(['graph_name'], as_index =False).agg('mean')

In [7]:
x= x.drop(['time','mcts_time'],axis = 1)
x = pd.merge(x,y,left_on='graph_name', right_on='graph_name')

In [9]:
x.to_csv('analysis.csv', index = False, encoding='utf-8')

In [8]:

import pandas as pd
import numpy as np

x = pd.read_csv(r'analysis.csv')
x['1st']=np.nan
x['2nd']=np.nan
x['3rd']=np.nan
x['4th']=np.nan
x['5th']=np.nan
x['winner_time']= np.nan
order = ['1st','2nd','3rd','4th','5th']
for i, row in x.iterrows():
    p=sorted(row[5:10])
    for i in range(5):
        for w in row:
            if p[i] == w:
                x.loc[x['graph_name']==row['graph_name'],order[i]] = row[row==w].index[0]
                if i == 0:
                    x.loc[x['graph_name']==row['graph_name'],'winner_time'] = w

x

,graph_name,mdfvs_size,v,e,diff,time_0,time_1,time_2,time_5,time_10,1st,2nd,3rd,4th,5th,winner_time
0,1410_e_049,35,62,344,27,35.429305,17.713489,20.191466,23.426836,34.270108,time_1,time_2,time_5,time_10,time_0,17.713489
1,1419_e_049,39,69,396,30,53.595896,37.149907,43.131311,38.511209,49.381287,time_1,time_5,time_2,time_10,time_0,37.149907
2,1611_e_053,34,51,304,17,25.778754,12.749171,14.500663,15.814576,23.597418,time_1,time_2,time_5,time_10,time_0,12.749171
3,1627_e_053,30,44,336,14,17.754756,13.674103,15.692257,20.905990,26.806900,time_1,time_2,time_0,time_5,time_10,13.674103
4,1644_e_053,31,44,337,13,18.525465,12.164283,13.843820,19.270255,28.386653,time_1,time_2,time_0,time_5,time_10,12.164283
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,6112_e_101,35,52,321,17,17.116497,16.266606,18.204713,21.838736,28.924612,time_1,time_0,time_2,time_5,time_10,16.266606
72,6126_e_101,40,66,406,26,65.476934,45.097112,47.322892,48.516731,47.499672,time_1,time_2,time_10,time_5,time_0,45.097112
73,6199_e_101,32,48,308,16,16.758519,7.315745,7.471738,11.862276,18.557207,time_1,time_2,time_5,time_0,time_10,7.315745
74,6229_e_101,38,57,392,19,24.157238,13.620380,13.571132,17.460701,24.879670,time_2,time_1,time_5,time_0,time_10,13.571132


In [9]:
x['diff_0_1'] = x['time_0']-x['time_1']

In [10]:
x

,graph_name,mdfvs_size,v,e,diff,time_0,time_1,time_2,time_5,time_10,1st,2nd,3rd,4th,5th,winner_time,diff_0_1
0,1410_e_049,35,62,344,27,35.429305,17.713489,20.191466,23.426836,34.270108,time_1,time_2,time_5,time_10,time_0,17.713489,17.715816
1,1419_e_049,39,69,396,30,53.595896,37.149907,43.131311,38.511209,49.381287,time_1,time_5,time_2,time_10,time_0,37.149907,16.445989
2,1611_e_053,34,51,304,17,25.778754,12.749171,14.500663,15.814576,23.597418,time_1,time_2,time_5,time_10,time_0,12.749171,13.029584
3,1627_e_053,30,44,336,14,17.754756,13.674103,15.692257,20.905990,26.806900,time_1,time_2,time_0,time_5,time_10,13.674103,4.080654
4,1644_e_053,31,44,337,13,18.525465,12.164283,13.843820,19.270255,28.386653,time_1,time_2,time_0,time_5,time_10,12.164283,6.361182
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71,6112_e_101,35,52,321,17,17.116497,16.266606,18.204713,21.838736,28.924612,time_1,time_0,time_2,time_5,time_10,16.266606,0.849891
72,6126_e_101,40,66,406,26,65.476934,45.097112,47.322892,48.516731,47.499672,time_1,time_2,time_10,time_5,time_0,45.097112,20.379822
73,6199_e_101,32,48,308,16,16.758519,7.315745,7.471738,11.862276,18.557207,time_1,time_2,time_5,time_0,time_10,7.315745,9.442774
74,6229_e_101,38,57,392,19,24.157238,13.620380,13.571132,17.460701,24.879670,time_2,time_1,time_5,time_0,time_10,13.571132,10.536858


In [11]:
print(x['1st'].value_counts())
print(x['2nd'].value_counts())
print(x['3rd'].value_counts())
print(x['4th'].value_counts())
print(x['5th'].value_counts())

time_1     47
time_2     21
time_5      4
time_10     2
time_0      2
Name: 1st, dtype: int64
time_2    46
time_1    19
time_0     6
time_5     5
Name: 2nd, dtype: int64
time_5     45
time_0     12
time_2      9
time_10     6
time_1      4
Name: 3rd, dtype: int64
time_10    32
time_5     22
time_0     16
time_1      6
Name: 4th, dtype: int64
time_0     40
time_10    36
Name: 5th, dtype: int64


In [16]:
print(x['time_0'].mean())
print(x['time_1'].mean())
print(x['time_2'].mean())
print(x['time_5'].mean())
print(x['time_10'].mean())
print('---------------')
print(x['diff_0_1'].mean())
print((x['time_0']-x['time_2']).mean())
print((x['time_0']-x['time_5']).mean())
print((x['time_0']-x['time_10']).mean())
print((x['time_0']-x['winner_time']).mean())

45.981562192236844
28.269386946315798
27.88346050094737
31.613785303815792
39.397360534868426
---------------
17.712175245921053
18.098101691289475
14.367776888421055
6.58420165736842
19.86632865581579


In [17]:
print((x['winner_time']).mean())

26.115233536421055
